In [ ]:
import cv2
import yt_dlp
import time
from datetime import datetime
from pathlib import Path
from ultralytics import YOLO

# ==============================================================================
# 1. CẤU HÌNH ĐƯỜNG DẪN & THAM SỐ
# ==============================================================================
# Link video YouTube cần xử lý
YOUTUBE_URL = "https://www.youtube.com/watch?v=LCauiIO0Bt4" 

# Đường dẫn Model
MODEL_PATH = Path(r"./best.pt")

# Các thư mục lưu trữ (Theo yêu cầu của bạn)
PATH_HELMET    = Path(r"./2_Detect_Helmet")
PATH_NO_HELMET = Path(r"./3_Detect_No_Helmet")
PATH_VIDEO_OUT = Path(r"./8_Video output")

# Cấu hình lưu ảnh & Detect
SAVE_FRAME_INTERVAL = 6   # Cứ 6 frame video thì xét LƯU ẢNH 1 lần (khoảng 5 ảnh/giây)
CONF_VIOLATION      = 0.1 # Độ nhạy bắt lỗi (thấp để bắt chặt)
CONF_SAFE           = 0.4 # Độ tin cậy an toàn
IOU_THRESHOLD       = 0.3

# Từ khóa nhận diện (Dựa trên model của bạn)
KW_UNSAFE = ['no-helmet', 'head', 'face', 'person', 'no_helmet']
KW_SAFE   = ['helmet', 'hardhat', 'safety-cap']

# ==============================================================================
# 2. HÀM HỖ TRỢ
# ==============================================================================
def setup_dirs():
    """Tạo các thư mục nếu chưa tồn tại"""
    for p in [PATH_HELMET, PATH_NO_HELMET, PATH_VIDEO_OUT]:
        p.mkdir(parents=True, exist_ok=True)
    print("📂 Đã kiểm tra và tạo xong các thư mục lưu trữ.")

def get_stream_url(url):
    """Lấy link stream trực tiếp từ YouTube"""
    print(f"📡 Đang kết nối YouTube: {url}...")
    ydl_opts = {'format': 'best[ext=mp4]', 'quiet': True}
    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(url, download=False)
            return info['url'], info['title']
    except Exception as e:
        print(f"❌ Lỗi kết nối: {e}")
        return None, None

# ==============================================================================
# 3. CHƯƠNG TRÌNH CHÍNH
# ==============================================================================
def run_realtime_detection():
    # A. Khởi tạo
    setup_dirs()
    
    if not MODEL_PATH.exists():
        print(f"❌ Không tìm thấy model tại: {MODEL_PATH}")
        return

    print(f"⏳ Đang tải model YOLOv8...")
    model = YOLO(str(MODEL_PATH))

    stream_url, video_title = get_stream_url(YOUTUBE_URL)
    if not stream_url: return

    cap = cv2.VideoCapture(stream_url)
    
    # Lấy thông số video nguồn
    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps <= 0: fps = 30
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Chuẩn bị file Video Output
    time_str = datetime.now().strftime("%Y%m%d_%H%M%S")
    safe_title = "".join([c for c in video_title if c.isalnum()]).strip()[:15]
    out_name = f"{safe_title}_{time_str}.mp4"
    out_path = PATH_VIDEO_OUT / out_name
    
    video_writer = cv2.VideoWriter(
        str(out_path), 
        cv2.VideoWriter_fourcc(*'mp4v'), 
        fps, 
        (width, height)
    )

    print(f"🎥 Đang xử lý & Hiển thị trực tiếp...")
    print(f"💾 Video sẽ lưu tại: {out_path}")
    print("⚡ Nhấn phím 'q' trên cửa sổ video để DỪNG chương trình.")

    frame_idx = 0
    
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("⚠️ Không đọc được frame hoặc hết video.")
                break
            
            frame_idx += 1

            # --- B1: DETECT ---
            results = model.predict(frame, conf=CONF_VIOLATION, iou=IOU_THRESHOLD, verbose=False)
            
            has_violation = False
            has_safe = False

            # --- B2: VẼ KHUNG & LOGIC ---
            for result in results:
                boxes = result.boxes
                for box in boxes:
                    cls_id = int(box.cls[0])
                    # Lấy tên class an toàn
                    if model.names:
                        cls_name = model.names[cls_id].lower().replace("_", "-")
                    else:
                        cls_name = "unknown"
                        
                    conf = float(box.conf[0])
                    x1, y1, x2, y2 = map(int, box.xyxy[0])

                    # Logic VI PHẠM (Đỏ) - Ưu tiên check trước
                    if any(k in cls_name for k in KW_UNSAFE) and conf >= CONF_VIOLATION:
                        has_violation = True
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 3)
                        cv2.putText(frame, f"NO-HELMET {conf:.2f}", (x1, y1 - 10), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
                    
                    # Logic AN TOÀN (Xanh)
                    elif any(k in cls_name for k in KW_SAFE) and conf >= CONF_SAFE:
                        # Vẫn vẽ xanh cho người an toàn kể cả khi trong hình có người vi phạm
                        if not has_violation: 
                            has_safe = True # Đánh dấu frame này là Safe (nếu chưa bị mark là Violation)
                        
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                        cv2.putText(frame, f"HELMET {conf:.2f}", (x1, y1 - 10), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

            # --- B3: HIỂN THỊ NGAY LẬP TỨC (REAL-TIME SHOW) ---
            # Resize nhẹ để hiển thị vừa màn hình nếu video quá lớn (tùy chọn)
            display_frame = cv2.resize(frame, (1280, 720)) if width > 1920 else frame
            cv2.imshow("AI Camera Monitor (Press 'q' to Stop)", display_frame)

            # --- B4: LƯU ẢNH (AUDIT) ---
            # Chỉ lưu theo nhịp (ví dụ mỗi 6 frame) để tránh nổ ổ cứng
            if frame_idx % SAVE_FRAME_INTERVAL == 0:
                img_name = f"Cap_{time_str}_{frame_idx}.jpg"
                
                if has_violation:
                    # Ưu tiên 1: Có vi phạm -> Vào folder No_Helmet
                    cv2.imwrite(str(PATH_NO_HELMET / img_name), frame)
                    # In log nhỏ để biết đang bắt lỗi
                    print(f"❌ Detect VI PHẠM -> {img_name}", end='\r')
                    
                elif has_safe:
                    # Ưu tiên 2: Không vi phạm, có mũ -> Vào folder Helmet
                    cv2.imwrite(str(PATH_HELMET / img_name), frame)

            # --- B5: GHI VIDEO ---
            video_writer.write(frame)

            # --- B6: CHECK PHÍM TẮT ---
            if cv2.waitKey(1) & 0xFF == ord('q'):
                print("\n⏹️ Đã nhận lệnh dừng từ bàn phím.")
                break

    except KeyboardInterrupt:
        print("\n⏹️ Dừng chương trình thủ công.")
    except Exception as e:
        print(f"\n❌ Có lỗi xảy ra: {e}")
    finally:
        cap.release()
        video_writer.release()
        cv2.destroyAllWindows()
        print(f"\n🎉 HOÀN TẤT!")
        print(f"📂 Video đã lưu tại: {out_path}")

if __name__ == "__main__":
    run_realtime_detection()

📂 Đã kiểm tra và tạo xong các thư mục lưu trữ.
⏳ Đang tải model YOLOv8...
📡 Đang kết nối YouTube: https://www.youtube.com/watch?v=LCauiIO0Bt4...


🎥 Đang xử lý & Hiển thị trực tiếp...
💾 Video sẽ lưu tại: D:\THS\06.1 Python\AI Camera\8_Video output\MakinoHungYenPr_20260104_143016.mp4
⚡ Nhấn phím 'q' trên cửa sổ video để DỪNG chương trình.
